In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from sklearn.model_selection import GroupKFold
import numpy as np

In [ ]:
class Generator(nn.Module):
  def __init__(self,input_length:int):
    super(Generator,self).__init__()
    self.dense_layer=nn.Linear(int(input_length),int(input_length))
    self.activation=nn.Sigmoid()
  def forward(self,x):
    return self.activation(self.dense_layer(x))

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,input_length:int):
    super(Discriminator,self).__init__()
    self.dense_layer=nn.Linear(int(input_length),1)
    self.activation=nn.Sigmoid()
  def forward(self,x):
    return self.activation(self.dense_layer(x))

In [ ]:
from typing import List

def create_binary_list_from_int(number: int) -> List[int]:
    return [int(x) for x in list(bin(number))[2:]]

In [ ]:
import math

In [ ]:
def generate_even_data(max_int: int, batch_size: int=16)->([List[int], List[List[int]]]):
    # Get the number of binary places needed to represent the maximum number
    max_length = int(math.log(max_int, 2))

    # Sample batch_size number of integers in range 0-max_int
    sampled_integers = np.random.randint(0, int(max_int / 2), batch_size)

    # create a list of labels all ones because all numbers are even
    labels = [1] * batch_size

    # Generate a list of binary numbers for training.
    data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]
    data=np.array(data)
    labels=np.array(labels)
    return labels, data

In [ ]:
def train(max_int: int = 128, batch_size: int = 16, training_steps: int = 500):
    input_length = int(math.log(max_int, 2))

    # Models
    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    # Optimizers
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
    discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

    # loss
    loss = nn.BCELoss()

    for i in range(training_steps):
        # zero the gradients on each iteration
        generator_optimizer.zero_grad()

        # Create noisy input for generator
        # Need float type instead of int
        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        random_noise=torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)
        generated_data1= generator(noise)
        
        if i%50==0:
          generated_data1=generated_data1.detach().numpy()
          generated_data1=(generated_data1>=0.5)*1
          x,y=generated_data1.shape
          generated_output=[]
          noise1=[]
          for j in range(x):
            sum=0
            for k in range(y):
              sum=sum+pow(2,y-1-k)*generated_data1[j][k]
            generated_output.append(sum)
          random_noise=random_noise.detach().numpy()
          #random_noise=(random>=0.5)*1
          u,v=random_noise.shape
          for j in range(u):
            sum=0
            for k in range(v):
              sum=sum+pow(2,v-1-k)*random_noise[j][k]
            noise1.append(sum)
          print(f'generated_output:{generated_output}')
          print(f'input:{noise1}')
        # Generate examples of even real data
        true_labels, true_data = generate_even_data(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_data = torch.tensor(true_data).float()
        #print(true_data.shape)
        #print(true_labels.unsqueeze(1).shape)
        # Train the generator
        # We invert the labels here and don't train the discriminator because we want the generator
        # to make things the discriminator classifies as true.
        generator_discriminator_out = discriminator(generated_data)
        #print(generator_discriminator_out.shape)
        generator_loss = loss(generator_discriminator_out, true_labels.unsqueeze(1))
        generator_loss.backward()
        generator_optimizer.step()

        # Train the discriminator on the true/generated data
        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels.unsqueeze(1))

        # add .detach() here think about this
        generator_discriminator_out = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size,1))
        discriminator_loss = (true_discriminator_loss + generator_discriminator_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()

In [ ]:
train(max_int= 2048, batch_size= 16, training_steps = 600)

generated_output:[808, 812, 873, 521, 1768, 1864, 617, 1801, 777, 333, 1837, 812, 808, 77, 589, 873]
input:[1425.0, 1355.0, 786.0, 1379.0, 275.0, 34.0, 2025.0, 1160.0, 1022.0, 1836.0, 840.0, 1529.0, 2046.0, 960.0, 76.0, 14.0]
generated_output:[940, 1421, 365, 1965, 1325, 1325, 1293, 941, 301, 1964, 813, 1929, 269, 1965, 1325, 1325]
input:[1615.0, 165.0, 525.0, 414.0, 911.0, 1834.0, 1972.0, 549.0, 92.0, 1423.0, 302.0, 295.0, 1962.0, 1762.0, 669.0, 1392.0]
generated_output:[1293, 269, 1421, 1453, 301, 300, 1421, 1325, 269, 1325, 1421, 1325, 1453, 301, 1453, 1421]
input:[1559.0, 1998.0, 741.0, 998.0, 2002.0, 1016.0, 1921.0, 56.0, 457.0, 1865.0, 980.0, 1702.0, 665.0, 1878.0, 533.0, 1321.0]
generated_output:[1420, 1420, 1452, 1420, 1293, 300, 1165, 1420, 1293, 1421, 268, 1420, 268, 1420, 1420, 1420]
input:[706.0, 1067.0, 549.0, 1450.0, 760.0, 1239.0, 243.0, 1710.0, 1237.0, 1555.0, 1415.0, 1215.0, 1480.0, 1005.0, 1561.0, 78.0]
generated_output:[1420, 1420, 396, 1420, 1420, 1420, 1420, 1420, 